# Simular Cabo Umbilical

Este Jupyter Notebook (kernel `julia`) prepara e roda alguns casos de simulação. O primeiro passo é importar os módulos e arquivos auxiliares.

In [1]:
include("param_cabo.jl");

import Combinatorics.combinations

In [2]:
@show Threads.nthreads()

Threads.nthreads() = 12


12

## Parâmetros do umbilical

Os condutores e índices correspondentes estão ilustrados na figura seguinte:

<img src="umbilical_condutores.png" alt="umbilical_condutores" width="600"/>

In [3]:
## Geometria do umbilical PT
rc = 1e-3 * ([9.6, 17.054, 18.054, 19.50])  # raios dos cabos SC [m]: condutor central, isolante, blindagem e isolante externo
ra = 1e-3 * ([48, 59, 65])  # raios da armadura [m]: interno, externo, capa
nx = 2  # número de condutores "ativos" em cada cabo SC (single core)

# distância do centro do umbilical ao centro das veias
d01 = rc[4] / cos(deg2rad(30))
d02 = d01
di = [d01, d02, d02]
theta_jk = (2 * pi) / 3  # ângulo entre cabos do umbilical

# permissividades elétricas
epsr_c = 3.31  # condutor central
epsr_b = 2.3  # blindagem, jaqueta da veia
epsr_a = 10  # última camada, que envolve a armadura

# resistividades elétricas [Ohm.m]
rho_c = 1.7241e-8  # condutor central
rho_b = 2.2e-7  # blindagem, jaqueta da veia
rho_a = 2.86e-8  # armadura

mur_a = 300  # permeabilidade magnética relativa da armadura
sig_s = 5.0  # condutividade elétrica do mar [S/m]
eps_s = 81.0  # permissividade elétrica relativa do mar

fases = [1, 3, 5]  # não alterar
blindagens = [2, 4, 6]  # não alterar
armadura = 7  # não alterar
condutores = [fases; blindagens; armadura]
nc1 = length(condutores)  # número de condutores
nc2 = nc1 * 2
nome_condutores = sort(Dict([
    [fases[i] => "fase_$(i)" for i in eachindex(fases)];
    [blindagens[i] => "blindagem_$(i)" for i in eachindex(blindagens)];
    [armadura => "armadura"]
]))
pares_falhas = collect(combinations(condutores, 2))
num_pares_falhas = length(pares_falhas)

21

## Parâmetros da simulação

Por causa da transformada numérica de Laplace utilizada, 20% dos tempos finais serão lixo numérico. Além disto, para minimizar o _zero padding_ (e oscilação numérica) da transformada de Fourier (FFTW), utilize um número de passos no tempo que seja potência de 2.

In [4]:
## Cálculo dos parâmetros e matrizes
## Tempo
dt = 2.5e-9  # passo no tempo [s]
# Usar número nt que seja potência de 2, para diminuir lixo numérico do zero-padding da FFT
nt = 1024 * 98  # número de passos no tempo
tempo = range(0, step=dt, length=nt)
tmax = tempo[end]
tempo_abre = 10e-6  # [s] que a chave desconecta a fonte do cabo
nt_trunc = Int(ceil(nt * 0.80))  # 20% dos tempos finais é lixo numérico
tmax_trunc = tempo[nt_trunc]  # tempo [s] final que será salvo

v_fonte_t = ones(nt)
#v_fonte_t = CSV.read("v_fonte_t.csv", DataFrame)  # pra ler de um CSV
R_chave = 50.0  # [Ω]
R_curto = 1e-6  # [Ω], tendência de causar instabilidade numérica para valores muito pequenos

1.0e-6

## Circuito

O circuito a ser simulado está ilustrado na figura seguinte. Ele consiste de dois segmentos de cabos (parâmetros distribuídos) com dois vetores de resistências: `R_falha_serie` que insere uma resistência em série nos condutores, indivudalmente, no ponto da falha, e `R_falha_shunt` que conecta as fases para suas blindagens e todas as blindagens entre si e estas para a armadura. Um valor negativo de `R_falha_serie[i]` significa sem falha série (curto-circuito) no condutor `i` e `R_falha_shunt[k]` significa que não há falha (resistência infinita) entre o par `k`.

O circuito de injeção consiste numa fonte de tensão em série com uma resistência `R_chave` e uma chave normal fechada que abre no tempo `tempo_abre`. Esse chaveamento é necessário, pois do contrário a fonte de tensão continuaria a forçar um valor no terminal, independentemente do valor de `R_chave`. Depois da chave há um vetor de resistências `R_emissor_shunt` que permanece conectada nos terminais emissor (excitação) durante toda a simulação. Um valor negativo de `R_emissor_shunt[i]` significa que não há resistência conectada no terminal `i`, isto é, equivale a uma resistência infinita.

Podem ser colocados um número arbitrário de segmentos de cabo. A falha é inserida no lado receptor (direita) do segmento correspondente à variável `segmento_falha`. O vetor booleano `aterrar_receptor` controla se o receptor de cada segmento tem as blindagens e armadura aterradas ou não.

<img src="circuito_umbilical.png" alt="circuito_umbilical" width="1000"/>

O bloca seguinte seção dá a correspondência entre o vetor `R_falha_shunt` e os pares de condutores.

In [5]:
println("Correspondência entre R_falha_shunt e par de condutores:")
for (i, pf) in enumerate(pares_falhas)
    c1 = nome_condutores[pf[1]]
    c2 = nome_condutores[pf[2]]
    println("R_falha_shunt[$(i)] => $(c1) para $(c2)")
end

Correspondência entre R_falha_shunt e par de condutores:
R_falha_shunt[1] => fase_1 para fase_2
R_falha_shunt[2] => fase_1 para fase_3
R_falha_shunt[3] => fase_1 para blindagem_1
R_falha_shunt[4] => fase_1 para blindagem_2
R_falha_shunt[5] => fase_1 para blindagem_3
R_falha_shunt[6] => fase_1 para armadura
R_falha_shunt[7] => fase_2 para fase_3
R_falha_shunt[8] => fase_2 para blindagem_1
R_falha_shunt[9] => fase_2 para blindagem_2
R_falha_shunt[10] => fase_2 para blindagem_3
R_falha_shunt[11] => fase_2 para armadura
R_falha_shunt[12] => fase_3 para blindagem_1
R_falha_shunt[13] => fase_3 para blindagem_2
R_falha_shunt[14] => fase_3 para blindagem_3
R_falha_shunt[15] => fase_3 para armadura
R_falha_shunt[16] => blindagem_1 para blindagem_2
R_falha_shunt[17] => blindagem_1 para blindagem_3
R_falha_shunt[18] => blindagem_1 para armadura
R_falha_shunt[19] => blindagem_2 para blindagem_3
R_falha_shunt[20] => blindagem_2 para armadura
R_falha_shunt[21] => blindagem_3 para armadura


# Rotina Principal

Pré-cálculo das matrizes de admitância nodal $Y_N$ dos segmentos, a qual depende do comprimento, tempo final `tmax` e número de passos no tempo `nt`. Resultados salvos em um arquivo HDF5. O cálculo só é realizado se não houver valor correspondente no arquivo pré-existente (se existir).

In [6]:
Lc = 2.5e3  # comprimento [m] total do cabo
comprimentos = [0.2, 0.4, 0.6, 0.8] .* Lc  # todos comprimentos [m] que serão necessários nas simulações
arquivo_matrizes = "umbilical_tripolar.h5"
precalc_matrizes(arquivo_matrizes, comprimentos, tmax, nt,
                 rc, ra, rho_a, rho_b, rho_c, epsr_a, epsr_b, epsr_c,
                 mur_a, theta_jk, di, sig_s, eps_s, nx)

Pré-carregar as matrizes $Y_N$ de todos os comprimentos que serão usados nas simulações abaixo.

In [12]:
sk, _ = laplace(ones(nt), tmax, nt)
nf = length(sk)  # número de frequências
comprimentos = [0.2, 0.4, 0.6, 0.8] .* Lc
num_comprimentos = length(comprimentos)
yn_comprimentos = zeros(ComplexF64, (nc2, nc2, nf, num_comprimentos))
for k = 1:num_comprimentos
    L1 = comprimentos[k]
    nome = "yn_" * string(round(L1, sigdigits=2))
    yn_comprimentos[:,:,:,k] .= reshape(h5read(arquivo_matrizes, nome), (nc2, nc2, nf))
end

## Sem falha

Cada resultado será salvo na pasta `"resultados"` com o nome `"caso_N.csv"`, onde `N` é um número. Se a pasta não existir ou se já existirem arquivos com estes nomes, acontecerá um erro.

In [13]:
segmento_falha = 1  # indice do segmento no qual o final tem a falha; não tem efeito no caso sem falha
aterrar_receptor = [false, true]  # blindagens e armadura aterradas no final do segmento?
lk = ReentrantLock()  # para evitar concorrência de dados
# Threads.@threads faz a execução paralela
ncaso = 0
Threads.@threads for i1 in eachindex(comprimentos)  # variar ponto da falha
    # selecionar comprimentos tal que `L1 + L2 == Lc`
    i2 = num_comprimentos - i1 + 1
    segmentos = comprimentos[[i1, i2]]
    yn_segmentos = yn_comprimentos[:,:,:,[i1,i2]]
    for terminal_fonte in fases  # fase a excitar
        R_emissor_shunt = fill(-1.0, nc1)  # tudo negativo: resistência infinita
        R_falha_shunt = fill(-1.0, num_pares_falhas)  # tudo negativo: resistência infinita
        R_falha_serie = fill(-1.0, nc1)  # tudo negativo: resistência nula
        vout_t = simular_cabo(yn_segmentos, R_curto, R_chave, R_emissor_shunt,
                              R_falha_shunt, R_falha_serie, segmento_falha,
                              pares_falhas, terminal_fonte, fases, blindagens,
                              armadura, v_fonte_t, tmax, nt, tempo_abre, aterrar_receptor)
        #
        lock(lk)  # somente um thread pode executar a seguinte seção por vez
        try
            global ncaso += 1
            arquivo_resultados = "resultados/caso_$(ncaso).csv"
            salvar_resultados(arquivo_resultados, vout_t, v_fonte_t, tempo, tempo_abre,
                              segmentos, segmento_falha, pares_falhas, terminal_fonte,
                              R_emissor_shunt, R_falha_serie, R_falha_shunt, R_chave,
                              nt_trunc, nome_condutores, aterrar_receptor)
        finally
            unlock(lk)  # libera para o próximo thread
        end
    end
end

## Falha shunt (paralela) entre condutores

Neste exemplo, só é simulada falha shunt em um único par:

In [14]:
ipar_falha = 3  # par envolvido na falha
println(nome_condutores[pares_falhas[ipar_falha][1]])
println(nome_condutores[pares_falhas[ipar_falha][2]])

fase_1
blindagem_1


In [15]:
Threads.@threads for i1 in eachindex(comprimentos)  # variar ponto da falha
    # selecionar comprimentos tal que `L1 + L2 == Lc`
    i2 = num_comprimentos - i1 + 1
    segmentos = comprimentos[[i1, i2]]
    yn_segmentos = yn_comprimentos[:,:,:,[i1,i2]]
    for terminal_fonte in fases  # fase a excitar
        for R in [1e-4, 1e-2, 1e2, 1e4]  # resistência [Ω] da falha
            R_emissor_shunt = fill(-1.0, nc1)  # tudo negativo: resistência infinita
            R_falha_shunt = fill(-1.0, num_pares_falhas)  # tudo negativo: resistência infinita
            R_falha_shunt[ipar_falha] = R
            R_falha_serie = fill(-1.0, nc1)  # tudo negativo: resistência nula
            vout_t = simular_cabo(yn_segmentos, R_curto, R_chave, R_emissor_shunt,
                                  R_falha_shunt, R_falha_serie, segmento_falha,
                                  pares_falhas, terminal_fonte, fases, blindagens,
                                  armadura, v_fonte_t, tmax, nt, tempo_abre, aterrar_receptor)
            #
            lock(lk)  # somente um thread pode executar a seguinte seção por vez
            try
                global ncaso += 1
                arquivo_resultados = "resultados/caso_$(ncaso).csv"
                salvar_resultados(arquivo_resultados, vout_t, v_fonte_t, tempo, tempo_abre,
                                  segmentos, segmento_falha, pares_falhas, terminal_fonte,
                                  R_emissor_shunt, R_falha_serie, R_falha_shunt, R_chave,
                                  nt_trunc, nome_condutores, aterrar_receptor)
            finally
                unlock(lk)  # libera para o próximo thread
            end
        end
    end
end

## Falha série (rompimento) entre condutores

Neste exemplo, só é simulada falha série em um único condutor.

In [16]:
cond_falha = fases[1]
Threads.@threads for i1 in eachindex(comprimentos)  # variar ponto da falha
    # selecionar comprimentos tal que `L1 + L2 == Lc`
    i2 = num_comprimentos - i1 + 1
    segmentos = comprimentos[[i1, i2]]
    yn_segmentos = yn_comprimentos[:,:,:,[i1,i2]]
    for terminal_fonte in fases  # fase a excitar
        for R in [1e-4, 1e-2, 1e2, 1e4]  # resistência [Ω] da falha
            R_emissor_shunt = fill(-1.0, nc1)  # tudo negativo: resistência infinita
            R_falha_shunt = fill(-1.0, num_pares_falhas)  # tudo negativo: resistência infinita
            R_falha_serie = fill(-1.0, nc1)  # tudo negativo: resistência nula
            R_falha_serie[cond_falha] = R
            vout_t = simular_cabo(yn_segmentos, R_curto, R_chave, R_emissor_shunt,
                                  R_falha_shunt, R_falha_serie, segmento_falha,
                                  pares_falhas, terminal_fonte, fases, blindagens,
                                  armadura, v_fonte_t, tmax, nt, tempo_abre, aterrar_receptor)
            #
            lock(lk)  # somente um thread pode executar a seguinte seção por vez
            try
                global ncaso += 1
                arquivo_resultados = "resultados/caso_$(ncaso).csv"
                salvar_resultados(arquivo_resultados, vout_t, v_fonte_t, tempo, tempo_abre,
                                  segmentos, segmento_falha, pares_falhas, terminal_fonte,
                                  R_emissor_shunt, R_falha_serie, R_falha_shunt, R_chave,
                                  nt_trunc, nome_condutores, aterrar_receptor)
            finally
                unlock(lk)  # libera para o próximo thread
            end
        end
    end
end